In [1]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 51.5 MB/s eta 0:00:00


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tensorflow as tf


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
!unzip /content/drive/MyDrive/MP_Data.zip -d /content/MP_Data

Streaming output truncated to the last 5000 lines.
  inflating: /content/MP_Data/MP_Data/Work/23/9.npy  
   creating: /content/MP_Data/MP_Data/Work/24/
  inflating: /content/MP_Data/MP_Data/Work/24/0.npy  
  inflating: /content/MP_Data/MP_Data/Work/24/1.npy  
  inflating: /content/MP_Data/MP_Data/Work/24/10.npy  
  inflating: /content/MP_Data/MP_Data/Work/24/11.npy  
  inflating: /content/MP_Data/MP_Data/Work/24/12.npy  
  inflating: /content/MP_Data/MP_Data/Work/24/13.npy  
  inflating: /content/MP_Data/MP_Data/Work/24/14.npy  
  inflating: /content/MP_Data/MP_Data/Work/24/15.npy  
  inflating: /content/MP_Data/MP_Data/Work/24/16.npy  
  inflating: /content/MP_Data/MP_Data/Work/24/17.npy  
  inflating: /content/MP_Data/MP_Data/Work/24/18.npy  
  inflating: /content/MP_Data/MP_Data/Work/24/19.npy  
  inflating: /content/MP_Data/MP_Data/Work/24/2.npy  
  inflating: /content/MP_Data/MP_Data/Work/24/3.npy  
  inflating: /content/MP_Data/MP_Data/Work/24/4.npy  
  inflating: /content/MP_Dat

In [5]:
%cd

/root


In [6]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [7]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [8]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [9]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

Extract keypoints

In [10]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

Setup folder for collection

In [11]:
# Path for exported data, numpy arrays
DATA_PATH = "/content/MP_Data/MP_Data"

# Actions that we try to detect
actions = np.array(['Sorry','Fast','Help','Love','Man'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 10

In [12]:
actions.shape[0]

5

preprocessing data

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [14]:
label_map = {label:num for num, label in enumerate(actions)}

In [15]:
label_map

{'Sorry': 0, 'Fast': 1, 'Help': 2, 'Love': 3, 'Man': 4}

In [16]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [17]:
np.array(sequences).shape

(150, 10, 258)

In [18]:
np.array(labels).shape

(150,)

In [19]:
X = np.array(sequences)

In [20]:
y = to_categorical(labels).astype(int)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [30]:
print(y_train.shape)

(120, 5)


In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import TensorBoard

In [23]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [24]:
from tensorflow.keras.models import load_model

In [32]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(10,258)))
model.add(LSTM(128,return_sequences = True,activation = 'relu'))
model.add(LSTM(64,return_sequences = False,activation = 'relu'))
model.add(Dense(64,activation = 'relu'))
model.add(Dense(32,activation = 'relu'))
model.add(Dense(actions.shape[0],activation = 'softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 10, 64)            82688     
                                                                 
 lstm_4 (LSTM)               (None, 10, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 5)                 165       
                                                                 
Total params: 237,317
Trainable params: 237,317
Non-tr

In [37]:
model.compile(optimizer = 'Adam',loss='categorical_crossentropy', metrics = ['accuracy'])

In [39]:
model.fit(X_train,y_train,epochs = 20, steps_per_epoch = 10, verbose=1,callbacks=[tb_callback])

Epoch 1/20
10/10 [==============================] - 0s 33ms/step - loss: 6.8358e-06 - accuracy: 1.0000
Epoch 2/20
10/10 [==============================] - 0s 27ms/step - loss: 5.7950e-06 - accuracy: 1.0000
Epoch 3/20
10/10 [==============================] - 0s 28ms/step - loss: 5.0063e-06 - accuracy: 1.0000
Epoch 4/20
10/10 [==============================] - 0s 28ms/step - loss: 4.4521e-06 - accuracy: 1.0000
Epoch 5/20
10/10 [==============================] - 0s 29ms/step - loss: 3.9773e-06 - accuracy: 1.0000
Epoch 6/20
10/10 [==============================] - 0s 30ms/step - loss: 3.4588e-06 - accuracy: 1.0000
Epoch 7/20
10/10 [==============================] - 0s 29ms/step - loss: 3.1350e-06 - accuracy: 1.0000
Epoch 8/20
10/10 [==============================] - 0s 29ms/step - loss: 2.8937e-06 - accuracy: 1.0000
Epoch 9/20
10/10 [==============================] - 0s 29ms/step - loss: 2.5251e-06 - accuracy: 1.0000
Epoch 10/20
10/10 [==============================] - 0s 29ms/step - loss:

In [40]:
%tensorboard --logdir Logs1

UsageError: Line magic function `%tensorboard` not found.


In [41]:
res = model.predict(X_test)

1/1 [==============================] - 0s 380ms/step


In [42]:
for i in range(5):
  print(actions[np.argmax(res[i])])

Man
Help
Man
Love
Love


In [43]:
for i in range(5):
    print(actions[np.argmax(y_test[i])])

Man
Help
Man
Love
Love


In [48]:
from scipy import stats

In [44]:
colors = []
for _ in range(len(label_map)):
    num = np.random.randint(0,256)
    colors.append((num,num,num))

In [49]:
colors

[(55, 55, 255),
 (250, 250, 255),
 (231, 231, 255),
 (217, 217, 255),
 (67, 67, 255)]

In [45]:
colors = []
for _ in range(len(label_map)):
    num = np.random.randint(0,256)
    colors.append((num,num,255))

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

    return output_frame

In [51]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30
#
#  ]
        sequence.append(keypoints)
        sequence = sequence[-10:]

        if len(sequence) == 10:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])


        # 3. Viz logic
            if res[np.argmax(res)] > threshold:
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5:
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)

        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()